In [1]:
# -*- coding: utf-8 -*-

from matplotlib import pyplot as plt
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torch
import os
from torch.optim import *
from torch import nn
from torch.nn import functional as F
from scipy import ndimage
import pandas as pd
from copy import deepcopy
import random
from sklearn.metrics import f1_score

random.seed(7)
torch.cuda.manual_seed(7)
torch.manual_seed(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(7)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [2]:
DATA_ROOT = '/beegfs/bva212/openmic-2018'
OPENMIC = np.load(os.path.join(DATA_ROOT, 'openmic-2018.npz'))
X, Y_true, Y_mask, sample_key = OPENMIC['X'], OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']

In [3]:
len_data = Y_mask.shape[0]
idx_train = np.random.choice(len_data, int(len_data*0.7), replace=False)
remain_set = set(np.arange(len_data))-set(idx_train)
idx_test = np.random.choice(list(remain_set), int(len_data*0.1), replace=False)
idx_val = list(remain_set-set(idx_test))

In [4]:
Y_mask_val = Y_mask[idx_val]
Y_mask_test = Y_mask[idx_test]

label_val = Y_true[idx_val]
label_test = Y_true[idx_test]

In [5]:
class ArrowOfTime(Dataset):

    def __init__(self, root_dir, files, weights, label):
        self.weights = weights
        self.device = device
        self.root_dir = root_dir
        self.files = files
        self.label = label

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        logscalogram = (np.load(self.root_dir + self.files[idx]+'_cqt.npy') - -24.3633)/14.2659
        weight = self.weights[idx]
        label = (self.label[idx] > 0.5).astype(int)
        return {'logscalogram': logscalogram[np.newaxis, :], 'label': label[np.newaxis, :], 'weight': weight[np.newaxis, :]}

filenames = []
root_dir = '/beegfs/bva212/openmic-2018/cqt_full/'

BATCH_SIZE = 32

def my_collate(batch):
    data = np.stack([item['logscalogram'] for item in batch])
    target = np.concatenate([item['label'] for item in batch],axis=0)
    weight = np.concatenate([item['weight'] for item in batch],axis=0)
    weight_sum = weight.sum(0)
    weight_sum = np.repeat(weight_sum[np.newaxis, :], len(batch), 0)
    weight_sum[weight_sum==0]=1
    weight = weight/weight_sum
    return [torch.from_numpy(data).float(), torch.from_numpy(target).float(), torch.from_numpy(weight).float()]


In [6]:
class snet2_jigsaw(nn.Module):

    def __init__(self, output_classes = 20):
        '''
        Create the 5 Conv Layer Sound Net network architecture as per the paper - https://arxiv.org/pdf/1610.09001.pdf
        '''
        super(snet2_jigsaw, self).__init__()

        self.conv_layers = nn.Sequential(nn.Conv2d(in_channels = 1, out_channels= 16, kernel_size = 5, stride = 2, padding = 5), 
                                nn.BatchNorm2d(num_features = 16), 
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 2, padding = 5),
                                nn.BatchNorm2d(32),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 5),
                                nn.BatchNorm2d(64),
                                nn.ReLU(inplace = True),
                                nn.AvgPool2d(kernel_size = 3),

                                nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 4, padding = 4),
                                nn.BatchNorm2d(128),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 4, padding = 4),
                                nn.BatchNorm2d(256),
                                nn.ReLU(inplace = True),
                                nn.AvgPool2d(kernel_size = 3),

                                nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 2, padding = 3),
                                nn.BatchNorm2d(512),
                                nn.ReLU(inplace = True),

                                nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3, stride = 2, padding = 3),
                                nn.BatchNorm2d(1024),
                                nn.ReLU(inplace = True),
                                nn.AdaptiveAvgPool2d(output_size = 1)
                                )
        self.mlp_layer = nn.Linear(1024, output_classes)
              
    def forward(self, input):
        out = self.conv_layers(input)
        out = self.mlp_layer(out.view(out.shape[0], -1))
        return out
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
        
# Function for testing the model
def test_model(loader, model):
    correct = 0
    total_loss = 0
    total = 0
    total_num = 0
    actual_arr = []
    predicted_arr = []
    weight_array = []
    model.eval()
    with torch.no_grad():
        for spectrogram, target, weight in loader:
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            predicted = (torch.sigmoid(outputs.data)>0.5).float()
            
            actual_arr.extend(target.view(1,-1).squeeze().numpy().astype(int).tolist())
            predicted_arr.extend(predicted.view(1,-1).squeeze().cpu().numpy().astype(int).tolist())
            
            total_loss += loss.item()
            total += weight_batch.shape[0]
            
            correct += ((weight_batch != 0).float()*(predicted.eq(target_batch.view_as(predicted)).float())).sum().item()
            total_num += (weight_batch != 0).sum().item()
            weight_array = np.concatenate((weight_array,(weight != 0).reshape(-1).numpy().astype(int)))
        accuracy = (100 * correct / total_num)
        return accuracy, f1_score(actual_arr, predicted_arr, average='micro', sample_weight = weight_array), total_loss/total

def train_model(train_loader, val_loader, model, optimizer, scheduler, num_epochs):
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []
    val_loss_list = []
    train_f1_list = []
    val_f1_list = []
    best_val_acc = 0
    for epoch in range(num_epochs):
        for spectrogram, target, weight in train_loader:
            model.train()
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            optimizer.zero_grad()
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            loss.backward()
            optimizer.step()
        train_acc, f1_score_train, train_loss = test_model(train_loader, model)
        val_acc, f1_score_val, val_loss = test_model(val_loader, model)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state_dict = deepcopy(model.state_dict())
        
        train_acc_list.append(train_acc)
        train_f1_list.append(f1_score_train)
        train_loss_list.append(train_loss)
        val_f1_list.append(f1_score_val)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)
        
        scheduler.step(val_acc)
        print("Epoch:{}".format(epoch+1))
        print("Validation Accuracy:{:.2f}, Validation F1:{:.2f}, Val Loss: {:.5f}".format(val_acc, f1_score_val, val_loss))
        print("Training Acc: {:.2f}, Training F1:{:.2f}, Train Loss: {:.5f}".format(train_acc, f1_score_train, train_loss))
    return train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict

In [7]:
import pickle
sizes = [10, 50, 250, 500, 1000, 1500]

Model_Data = pickle.load(open('List_indices_data_size_exp.pkl', 'rb'))

Val_dataset = ArrowOfTime(root_dir, sample_key[idx_val], Y_mask_val, label_val)
Val_loader = torch.utils.data.DataLoader(dataset = Val_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = True,
                                        collate_fn = my_collate)

Test_dataset = ArrowOfTime(root_dir, sample_key[idx_test], Y_mask_test, label_test)
Test_loader = torch.utils.data.DataLoader(dataset = Test_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = True,
                                        collate_fn = my_collate)

results_dict = {}

for i in range(len(sizes)):
    if sizes[i] != 1500:
        idx_train_new = Model_Data[i]
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[idx_train]
        label_train = Y_true[idx_train]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    weights = torch.load('/beegfs/bva212/capstone/new_model/checkpoint_model.pth')['modelStateDict']
    req_weights = {key[7:]:val for key, val in weights.items()}
    model = snet2_jigsaw(2)
    model.load_state_dict(req_weights)
    model.mlp_layer = nn.Linear(1024, 20)
    model.to(device)
    learning_rate = 10**(-2)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('Size_Exp_Flip_Wt_Results.pkl'.format(sizes[i]),'wb'))

Epoch:1
Validation Accuracy:55.53, Validation F1:0.56, Val Loss: 16.48911
Training Acc: 57.21, Training F1:0.57, Train Loss: 16.50494
Epoch:2
Validation Accuracy:58.75, Validation F1:0.59, Val Loss: 1.08304
Training Acc: 60.14, Training F1:0.60, Train Loss: 0.76670
Epoch:3
Validation Accuracy:64.36, Validation F1:0.64, Val Loss: 0.49380
Training Acc: 62.84, Training F1:0.63, Train Loss: 0.49575
Epoch:4
Validation Accuracy:61.35, Validation F1:0.61, Val Loss: 0.40122
Training Acc: 66.22, Training F1:0.66, Train Loss: 0.41026
Epoch:5
Validation Accuracy:59.86, Validation F1:0.60, Val Loss: 0.40197
Training Acc: 63.51, Training F1:0.64, Train Loss: 0.40662
Epoch     5: reducing learning rate of group 0 to 1.0000e-03.
Epoch:6
Validation Accuracy:59.07, Validation F1:0.59, Val Loss: 0.40637
Training Acc: 62.39, Training F1:0.62, Train Loss: 0.43515
Epoch:7
Validation Accuracy:58.69, Validation F1:0.59, Val Loss: 0.40660
Training Acc: 63.74, Training F1:0.64, Train Loss: 0.39958
Epoch:8
Vali

Epoch:10
Validation Accuracy:67.66, Validation F1:0.68, Val Loss: 0.35897
Training Acc: 71.40, Training F1:0.71, Train Loss: 0.33279
Epoch:11
Validation Accuracy:69.27, Validation F1:0.69, Val Loss: 0.35586
Training Acc: 71.93, Training F1:0.72, Train Loss: 0.33585
Epoch:12
Validation Accuracy:67.39, Validation F1:0.67, Val Loss: 0.35500
Training Acc: 70.68, Training F1:0.71, Train Loss: 0.33914
Epoch:13
Validation Accuracy:68.85, Validation F1:0.69, Val Loss: 0.34437
Training Acc: 72.42, Training F1:0.72, Train Loss: 0.32059
Epoch    13: reducing learning rate of group 0 to 1.0000e-04.
Epoch:14
Validation Accuracy:68.60, Validation F1:0.69, Val Loss: 0.35958
Training Acc: 71.06, Training F1:0.71, Train Loss: 0.34301
Epoch:15
Validation Accuracy:69.62, Validation F1:0.70, Val Loss: 0.33600
Training Acc: 73.24, Training F1:0.73, Train Loss: 0.31931
Epoch:16
Validation Accuracy:70.12, Validation F1:0.70, Val Loss: 0.33456
Training Acc: 73.96, Training F1:0.74, Train Loss: 0.30401
Epoch:1

Epoch:19
Validation Accuracy:70.93, Validation F1:0.71, Val Loss: 0.32622
Training Acc: 70.60, Training F1:0.71, Train Loss: 0.32449
Epoch:20
Validation Accuracy:70.97, Validation F1:0.71, Val Loss: 0.32419
Training Acc: 70.90, Training F1:0.71, Train Loss: 0.32242
Epoch:21
Validation Accuracy:71.14, Validation F1:0.71, Val Loss: 0.32454
Training Acc: 71.17, Training F1:0.71, Train Loss: 0.31897
Epoch:22
Validation Accuracy:70.44, Validation F1:0.70, Val Loss: 0.32237
Training Acc: 70.98, Training F1:0.71, Train Loss: 0.32435
Epoch:23
Validation Accuracy:71.48, Validation F1:0.71, Val Loss: 0.32505
Training Acc: 71.24, Training F1:0.71, Train Loss: 0.32177
Epoch:24
Validation Accuracy:70.79, Validation F1:0.71, Val Loss: 0.32671
Training Acc: 71.06, Training F1:0.71, Train Loss: 0.32279
Epoch:25
Validation Accuracy:71.42, Validation F1:0.71, Val Loss: 0.32442
Training Acc: 71.03, Training F1:0.71, Train Loss: 0.32148
Epoch    25: reducing learning rate of group 0 to 1.0000e-04.
Epoch:2

Epoch    26: reducing learning rate of group 0 to 1.0000e-07.
Epoch:27
Validation Accuracy:73.86, Validation F1:0.74, Val Loss: 0.30046
Training Acc: 74.19, Training F1:0.74, Train Loss: 0.29287
Epoch:28
Validation Accuracy:73.93, Validation F1:0.74, Val Loss: 0.29939
Training Acc: 74.28, Training F1:0.74, Train Loss: 0.29089
Epoch:29
Validation Accuracy:73.88, Validation F1:0.74, Val Loss: 0.29824
Training Acc: 74.24, Training F1:0.74, Train Loss: 0.29080
Epoch:30
Validation Accuracy:73.82, Validation F1:0.74, Val Loss: 0.29774
Training Acc: 74.20, Training F1:0.74, Train Loss: 0.29038
Epoch    30: reducing learning rate of group 0 to 1.0000e-08.
Epoch:31
Validation Accuracy:73.70, Validation F1:0.74, Val Loss: 0.29675
Training Acc: 74.23, Training F1:0.74, Train Loss: 0.29401
Epoch:32
Validation Accuracy:73.83, Validation F1:0.74, Val Loss: 0.29549
Training Acc: 74.22, Training F1:0.74, Train Loss: 0.29368
Epoch:33
Validation Accuracy:73.89, Validation F1:0.74, Val Loss: 0.29741
Trai

Epoch    37: reducing learning rate of group 0 to 1.0000e-04.
Epoch:38
Validation Accuracy:74.32, Validation F1:0.74, Val Loss: 0.29462
Training Acc: 75.18, Training F1:0.75, Train Loss: 0.28228
Epoch:39
Validation Accuracy:75.80, Validation F1:0.76, Val Loss: 0.27755
Training Acc: 76.36, Training F1:0.76, Train Loss: 0.27337
Epoch:40
Validation Accuracy:75.94, Validation F1:0.76, Val Loss: 0.28065
Training Acc: 76.43, Training F1:0.76, Train Loss: 0.27362
Epoch:41
Validation Accuracy:75.93, Validation F1:0.76, Val Loss: 0.27754
Training Acc: 76.58, Training F1:0.77, Train Loss: 0.27048
Epoch:42
Validation Accuracy:76.00, Validation F1:0.76, Val Loss: 0.28177
Training Acc: 76.58, Training F1:0.77, Train Loss: 0.27279
Epoch:43
Validation Accuracy:75.90, Validation F1:0.76, Val Loss: 0.27858
Training Acc: 76.63, Training F1:0.77, Train Loss: 0.27197
Epoch:44
Validation Accuracy:76.15, Validation F1:0.76, Val Loss: 0.27802
Training Acc: 76.73, Training F1:0.77, Train Loss: 0.26925
Epoch:4

Epoch:46
Validation Accuracy:73.70, Validation F1:0.74, Val Loss: 0.30203
Training Acc: 73.80, Training F1:0.74, Train Loss: 0.29752
Epoch:47
Validation Accuracy:73.82, Validation F1:0.74, Val Loss: 0.30692
Training Acc: 73.67, Training F1:0.74, Train Loss: 0.29732
Epoch:48
Validation Accuracy:73.77, Validation F1:0.74, Val Loss: 0.30256
Training Acc: 73.81, Training F1:0.74, Train Loss: 0.29613
Epoch:49
Validation Accuracy:73.75, Validation F1:0.74, Val Loss: 0.30130
Training Acc: 73.83, Training F1:0.74, Train Loss: 0.30013
Epoch:50
Validation Accuracy:73.83, Validation F1:0.74, Val Loss: 0.30371
Training Acc: 73.77, Training F1:0.74, Train Loss: 0.29708


In [8]:
results_dict = {}
model1 = snet2_jigsaw(20).to(device)
model = snet2_jigsaw(20).to(device)

for i in range(len(sizes)):
    if sizes[i] != 1500:
        idx_train_new = Model_Data[i]
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[idx_train]
        label_train = Y_true[idx_train]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    model.load_state_dict(model1.state_dict())
    learning_rate = 10**(-5)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('Size_Exp_Random_Wt_NoLinear_Results_size_{}.pkl'.format(sizes[i]),'wb'))

Epoch:1
Validation Accuracy:50.51, Validation F1:0.51, Val Loss: 0.41773
Training Acc: 49.77, Training F1:0.50, Train Loss: 0.44722
Epoch:2
Validation Accuracy:49.83, Validation F1:0.50, Val Loss: 0.41789
Training Acc: 47.97, Training F1:0.48, Train Loss: 0.43928
Epoch:3
Validation Accuracy:51.33, Validation F1:0.51, Val Loss: 0.41623
Training Acc: 52.48, Training F1:0.52, Train Loss: 0.43037
Epoch:4
Validation Accuracy:55.09, Validation F1:0.55, Val Loss: 0.41488
Training Acc: 60.14, Training F1:0.60, Train Loss: 0.43924
Epoch:5
Validation Accuracy:59.10, Validation F1:0.59, Val Loss: 0.40996
Training Acc: 70.72, Training F1:0.71, Train Loss: 0.41331
Epoch:6
Validation Accuracy:62.25, Validation F1:0.62, Val Loss: 0.40318
Training Acc: 74.55, Training F1:0.75, Train Loss: 0.40177
Epoch:7
Validation Accuracy:63.86, Validation F1:0.64, Val Loss: 0.39445
Training Acc: 78.38, Training F1:0.78, Train Loss: 0.39030
Epoch:8
Validation Accuracy:64.63, Validation F1:0.65, Val Loss: 0.39237
Tra

Epoch:12
Validation Accuracy:72.25, Validation F1:0.72, Val Loss: 0.33041
Training Acc: 79.90, Training F1:0.80, Train Loss: 0.29430
Epoch:13
Validation Accuracy:72.41, Validation F1:0.72, Val Loss: 0.32628
Training Acc: 80.19, Training F1:0.80, Train Loss: 0.28511
Epoch:14
Validation Accuracy:72.75, Validation F1:0.73, Val Loss: 0.32451
Training Acc: 80.68, Training F1:0.81, Train Loss: 0.28072
Epoch:15
Validation Accuracy:72.69, Validation F1:0.73, Val Loss: 0.33015
Training Acc: 81.01, Training F1:0.81, Train Loss: 0.27547
Epoch:16
Validation Accuracy:72.83, Validation F1:0.73, Val Loss: 0.32656
Training Acc: 82.08, Training F1:0.82, Train Loss: 0.26826
Epoch:17
Validation Accuracy:72.93, Validation F1:0.73, Val Loss: 0.32313
Training Acc: 81.88, Training F1:0.82, Train Loss: 0.26687
Epoch:18
Validation Accuracy:72.95, Validation F1:0.73, Val Loss: 0.32544
Training Acc: 83.19, Training F1:0.83, Train Loss: 0.25897
Epoch:19
Validation Accuracy:73.02, Validation F1:0.73, Val Loss: 0.3

Epoch:23
Validation Accuracy:79.38, Validation F1:0.79, Val Loss: 0.26798
Training Acc: 89.40, Training F1:0.89, Train Loss: 0.18110
Epoch:24
Validation Accuracy:79.44, Validation F1:0.79, Val Loss: 0.26071
Training Acc: 90.90, Training F1:0.91, Train Loss: 0.17040
Epoch:25
Validation Accuracy:80.09, Validation F1:0.80, Val Loss: 0.25901
Training Acc: 91.88, Training F1:0.92, Train Loss: 0.16493
Epoch:26
Validation Accuracy:80.06, Validation F1:0.80, Val Loss: 0.25765
Training Acc: 91.39, Training F1:0.91, Train Loss: 0.16099
Epoch:27
Validation Accuracy:80.46, Validation F1:0.80, Val Loss: 0.25160
Training Acc: 92.73, Training F1:0.93, Train Loss: 0.15069
Epoch:28
Validation Accuracy:80.37, Validation F1:0.80, Val Loss: 0.25256
Training Acc: 92.42, Training F1:0.92, Train Loss: 0.14458
Epoch:29
Validation Accuracy:80.29, Validation F1:0.80, Val Loss: 0.25514
Training Acc: 94.00, Training F1:0.94, Train Loss: 0.14036
Epoch    29: reducing learning rate of group 0 to 1.0000e-06.
Epoch:3

Epoch    33: reducing learning rate of group 0 to 1.0000e-06.
Epoch:34
Validation Accuracy:84.61, Validation F1:0.85, Val Loss: 0.21692
Training Acc: 97.94, Training F1:0.98, Train Loss: 0.07126
Epoch:35
Validation Accuracy:85.35, Validation F1:0.85, Val Loss: 0.20348
Training Acc: 99.14, Training F1:0.99, Train Loss: 0.06081
Epoch:36
Validation Accuracy:85.43, Validation F1:0.85, Val Loss: 0.20783
Training Acc: 99.21, Training F1:0.99, Train Loss: 0.05820
Epoch:37
Validation Accuracy:85.43, Validation F1:0.85, Val Loss: 0.20516
Training Acc: 99.26, Training F1:0.99, Train Loss: 0.05856
Epoch:38
Validation Accuracy:85.55, Validation F1:0.86, Val Loss: 0.19865
Training Acc: 99.28, Training F1:0.99, Train Loss: 0.05756
Epoch:39
Validation Accuracy:85.32, Validation F1:0.85, Val Loss: 0.20159
Training Acc: 99.26, Training F1:0.99, Train Loss: 0.05631
Epoch:40
Validation Accuracy:85.38, Validation F1:0.85, Val Loss: 0.19948
Training Acc: 99.25, Training F1:0.99, Train Loss: 0.05752
Epoch  

Epoch:43
Validation Accuracy:90.94, Validation F1:0.91, Val Loss: 0.13848
Training Acc: 99.91, Training F1:1.00, Train Loss: 0.02351
Epoch:44
Validation Accuracy:91.04, Validation F1:0.91, Val Loss: 0.14077
Training Acc: 99.91, Training F1:1.00, Train Loss: 0.02260
Epoch:45
Validation Accuracy:90.96, Validation F1:0.91, Val Loss: 0.13774
Training Acc: 99.91, Training F1:1.00, Train Loss: 0.02286
Epoch:46
Validation Accuracy:91.06, Validation F1:0.91, Val Loss: 0.14428
Training Acc: 99.91, Training F1:1.00, Train Loss: 0.02330
Epoch:47
Validation Accuracy:90.98, Validation F1:0.91, Val Loss: 0.14479
Training Acc: 99.92, Training F1:1.00, Train Loss: 0.02282
Epoch:48
Validation Accuracy:91.00, Validation F1:0.91, Val Loss: 0.14194
Training Acc: 99.91, Training F1:1.00, Train Loss: 0.02284
Epoch:49
Validation Accuracy:91.02, Validation F1:0.91, Val Loss: 0.13622
Training Acc: 99.92, Training F1:1.00, Train Loss: 0.02257
Epoch:50
Validation Accuracy:90.88, Validation F1:0.91, Val Loss: 0.1

In [12]:
mean = 0.
std = 0.
for images, _, _ in Train_loader:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images1 = images.reshape(batch_samples, images.size(1), -1)
    mean += images1.mean(2).sum(0)
    std += images1.std(2).sum(0)
    
mean /= len(Train_loader.dataset)
std /= len(Train_loader.dataset)

In [13]:
print(mean, std)

tensor([-24.3633]) tensor([14.2659])


In [65]:

model = conv_net().to(device)
model.load_state_dict(reqd_weights)

<All keys matched successfully>

In [9]:
pickle.dump(results_dict, open('Size_Exp_Jigsaw_Wt_Results.pkl'.format(sizes[i]),'wb'))

In [10]:
results_dict.keys()

dict_keys([10, 50, 250, 500])

In [19]:
import torchvision.models as models
resnet18 = models.resnet18().to(device)
alexnet = models.alexnet().to(device)
vgg16 = models.vgg16().to(device)
squeezenet = models.squeezenet1_0().to(device)
densenet = models.densenet161().to(device)
inception = models.inception_v3().to(device)


In [12]:
import numpy as np
a = np.array([1,2])
b = np.repeat(a[np.newaxis,:],2,0)
(b!=1).reshape(-1).astype(int)

array([0, 1, 0, 1])